In [14]:
import pandas as pd
import networkx as nx
import os.path as osp

import torch
import torch_geometric
from torch_geometric.data import Dataset, download_url
from torch_geometric.utils.convert import from_networkx
import numpy as np

from gensim.models import Word2Vec

from torch.nn import Parameter
import torch.nn.functional as F
from torch_geometric.nn import GCNConv,Linear,RGCNConv,ChebConv
from torch_geometric.nn import GAE, Node2Vec,VGAE
from torch.utils.tensorboard import SummaryWriter
from torch_geometric.nn.models.autoencoder import ARGVA

from torch_geometric.transforms import RandomLinkSplit
from tqdm import tqdm

In [2]:
torch.manual_seed(0)

In [4]:
df=pd.read_csv('PPI.csv')
G=nx.from_pandas_edgelist(df,'Official Symbol Interactor A','Official Symbol Interactor B' )

#load the model
model=Word2Vec.load('EMBEDDING_MODEL_FILENAME')
x=torch.tensor(model.wv.get_normed_vectors())
#x.shape

for n in G.nodes():
    G.nodes[n]['x']=np.array(model.wv.get_vector(n))
    
pyg_graph = from_networkx(G)    

c:\users\calni\appdata\local\programs\python\python39\lib\site-packages\torch_geometric\utils\convert.py:178: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ..\torch\csrc\utils\tensor_new.cpp:201.)
  data[key] = torch.tensor(value)


In [5]:
transform = RandomLinkSplit(is_undirected=False,split_labels=True,
                      neg_sampling_ratio=1.0,
                      key = "edge_label",
                      disjoint_train_ratio=0,
                      num_val =0)
train_data, val_data, test_data = transform(pyg_graph)

## GAE with Linear Decoder (AUC 0.93)

In [7]:
class GCNEncoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GCNEncoder, self).__init__()
        self.conv1 = GCNConv(in_channels, 2 * out_channels, cached=True) # cached only for transductive learning
        self.conv2 = GCNConv(2 * out_channels, out_channels, cached=True) # cached only for transductive learning

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x
    
class GCNDecoder(torch.nn.Module):
    def __init__(self, latent_dim):
        super(GCNDecoder, self).__init__()
        self.lin1 = Linear(latent_dim,latent_dim)
        self.lin2 = Linear(latent_dim,latent_dim//2)
        self.lin3 = Linear(latent_dim//2,1)

    def forward(self, z, edge_index, sigmoid=True):

        z = (z[edge_index[0]] * z[edge_index[1]])#.sum(dim=1)
        z = self.lin1(z).relu()
        z = self.lin2(z).relu()
        z = self.lin3(z)
        z=z.squeeze()
        
        return torch.sigmoid(z) if sigmoid else value    

In [8]:
def train(data):
    model.train()
    optimizer.zero_grad()
    z = model.encode(data.x, data.edge_index)
    pos_edge_index=data.pos_edge_label_index
    neg_edge_index=data.neg_edge_label_index
    loss = model.recon_loss(z, pos_edge_index,neg_edge_index) 
    loss.backward()
    optimizer.step()
    return float(loss)


def test(data):
    model.eval()
    with torch.no_grad():
        z = model.encode(data.x,data.edge_index)
        pos_edge_index=data.pos_edge_label_index
        neg_edge_index=data.neg_edge_label_index
    return model.test(z, pos_edge_index, neg_edge_index)

In [9]:
# parameters
out_channels = 20   #embedding 
num_features = train_data.x.shape[1] 
epochs = 100


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# model
model = GAE(GCNEncoder(num_features, out_channels),GCNDecoder(out_channels))
model = model.to(device)

# inizialize the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [10]:
writer = SummaryWriter('runs_5/GAE+dec_experiment'+'20d_100_epochs')

for epoch in tqdm(range(1, epochs + 1)):
    loss = train(train_data)
    auc, ap = test(test_data)
    #print('Epoch: {:03d}, AUC: {:.4f}, AP: {:.4f}'.format(epoch, auc, ap))
    
    writer.add_scalar('loss train',loss,epoch)
    writer.add_scalar('auc train',auc,epoch) 
    writer.add_scalar('ap train',ap,epoch) 

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [05:05<00:00,  3.05s/it]


## DeepGAE with Linear Decoder

In [11]:
class DeepGCNEncoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DeepGCNEncoder, self).__init__()
        self.conv1 = GCNConv(in_channels, 2 * out_channels, cached=True)
        self.conv2 = GCNConv(2 * out_channels, 2 * out_channels, cached=True)
        self.conv3 = GCNConv(2 * out_channels, out_channels, cached=True)

    def forward(self, x, edge_index,edge_weight=None):
        x = self.conv1(x, edge_index,edge_weight=edge_weight).relu()
        x = self.conv2(x, edge_index,edge_weight=edge_weight).relu()
        return self.conv3(x, edge_index,edge_weight=edge_weight)

In [12]:
# parameters
out_channels = 20   #embedding 
num_features = train_data.x.shape[1] 
epochs = 100


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# model
model = GAE(DeepGCNEncoder(num_features, out_channels),GCNDecoder(out_channels))
model = model.to(device)

# inizialize the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [13]:
writer = SummaryWriter('runs_5/DeepGAE+dec_experiment'+'20d_100_epochs')

for epoch in tqdm(range(1, epochs + 1)):
    loss = train(train_data)
    auc, ap = test(test_data)
    #print('EpoDeprecationWarning: {:03d}, AUC: {:.4f}, AP: {:.4f}'.format(epoch, auc, ap))
    
    writer.add_scalar('loss train',loss,epoch)
    writer.add_scalar('auc train',auc,epoch) 
    writer.add_scalar('ap train',ap,epoch) 

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [07:13<00:00,  4.33s/it]


## GAE with ChebConv (K=3 e K=5)

In [28]:
class ChebConvEncoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ChebConvEncoder, self).__init__()
        self.conv1 = ChebConv(in_channels, 2 * out_channels, K=5)  
        self.conv2 = ChebConv(2 * out_channels, out_channels, K=5) 

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x
    
class GCNDecoder(torch.nn.Module):
    def __init__(self, latent_dim):
        super(GCNDecoder, self).__init__()
        self.lin1 = Linear(latent_dim,latent_dim)
        self.lin2 = Linear(latent_dim,latent_dim//2)
        self.lin3 = Linear(latent_dim//2,1)

    def forward(self, z, edge_index, sigmoid=True):

        z = (z[edge_index[0]] * z[edge_index[1]])#.sum(dim=1)
        z = self.lin1(z).relu()
        z = self.lin2(z).relu()
        z = self.lin3(z)
        z=z.squeeze()
        
        return torch.sigmoid(z) if sigmoid else value

In [29]:
# parameters
out_channels = 20   #embedding 
num_features = train_data.x.shape[1] 
epochs = 100


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# model
model = GAE(ChebConvEncoder(num_features, out_channels),GCNDecoder(out_channels))
model = model.to(device)

# inizialize the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [30]:
writer = SummaryWriter('runs_5/ChebGAEk5+dec_experiment'+'20d_100_epochs')

for epoch in tqdm(range(1, epochs + 1)):
    loss = train(train_data)
    auc, ap = test(test_data)
    #print('EpoDeprecationWarning: {:03d}, AUC: {:.4f}, AP: {:.4f}'.format(epoch, auc, ap))
    
    writer.add_scalar('loss train',loss,epoch)
    writer.add_scalar('auc train',auc,epoch) 
    writer.add_scalar('ap train',ap,epoch) 

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [13:26<00:00,  8.06s/it]


## DeepGAE with ChebConv

In [31]:
class DeepChebConvEncoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DeepChebConvEncoder, self).__init__()
        self.conv1 = ChebConv(in_channels, 2 * out_channels,K=3)
        self.conv2 = ChebConv(2 * out_channels, 2 * out_channels,K=3)
        self.conv3 = ChebConv(2 * out_channels, out_channels,K=3)

    def forward(self, x, edge_index,edge_weight=None):
        x = self.conv1(x, edge_index,edge_weight=edge_weight).relu()
        x = self.conv2(x, edge_index,edge_weight=edge_weight).relu()
        return self.conv3(x, edge_index,edge_weight=edge_weight)

In [32]:
# parameters
out_channels = 20   #embedding 
num_features = train_data.x.shape[1] 
epochs = 100


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# model
model = GAE(DeepChebConvEncoder(num_features, out_channels),GCNDecoder(out_channels))
model = model.to(device)

# inizialize the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [33]:
writer = SummaryWriter('runs_5/DeepChebGAEk5+dec_experiment'+'20d_100_epochs')

for epoch in tqdm(range(1, epochs + 1)):
    loss = train(train_data)
    auc, ap = test(test_data)
    #print('EpoDeprecationWarning: {:03d}, AUC: {:.4f}, AP: {:.4f}'.format(epoch, auc, ap))
    
    writer.add_scalar('loss train',loss,epoch)
    writer.add_scalar('auc train',auc,epoch) 
    writer.add_scalar('ap train',ap,epoch) 

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [13:04<00:00,  7.84s/it]
